In [1]:
import pandas as pd
import json
import re
import numpy as np

In [2]:
lake = pd.read_csv("lake_info/lake_information.csv", converters={"lake_id": str})
length = pd.read_csv("fish_length_info/fish_length.csv", converters={"lake_id": str, "survey_id": str})
catch = pd.read_csv("fish_catch_info/fish_catch.csv", converters={"lake_id": str, "survey_id": str})
was = pd.read_csv("was_info/water_access_information.csv")

C:\Users\lukew\AppData\Local\Temp\ipykernel_1652\3492058089.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  catch = pd.read_csv("fish_catch_info/fish_catch.csv", converters={"lake_id": str, "survey_id": str})


In [3]:
#add methods
#sort and format year survey data then convert to datetime
def survey_date_cleaner(dfseries):
    dfseries = dfseries.copy()
    for _ in range(len(dfseries)):
        dfseries[_] = dfseries[_].replace("/","-")
        if matches := re.search(r"^(\d?\d{1})-(\d?\d{1})-(\d{4})$", dfseries[_], re.IGNORECASE):
            dfseries[_] = f"{matches.group(3)}-{matches.group(2)}-{matches.group(1)}"
        try:
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
        except ValueError:
            if matches := re.search(r"^(\d{4})-(\d?\d{1})-(\d?\d{1})$", dfseries[_], re.IGNORECASE):
                dfseries[_] = f"{matches.group(1)}-{matches.group(3)}-{matches.group(2)}"
            dfseries[_] = pd.to_datetime(dfseries[_],format="%Y-%m-%d")
    dfseries = pd.to_datetime(dfseries,format="%Y-%m-%d")
    return dfseries

#input CPUE column data series
#drop strings from cpue data series and convert to int(found some "? marks")
def cpue_cleaner(dfseries):
    i=0
    for _ in dfseries:
        try:
            np.float64(_)
            i = i + 1
        except ValueError:
            print(i, _,"valueerror")
            dfseries = dfseries.drop([i])
            i = i + 1
    dfseries = pd.to_numeric(dfseries)
    return dfseries

#input fish_count column data series
#returns fish_count column in dataframe as a list of individual integer measurements e.g. [0,0,1,1,1,2,3,3,4]
def fish_length_cleaner(dfseries):
    fish_lengths = []
    #scrub each row string and convert to list of lengths sampled
    for row in dfseries:
        #clean up string of list syntax
        sample = row.replace("]","").replace("[","").replace(" ","").split(",")
        #breakdown string into list of lengths sampled for row
        sample_lengths = []
        for n in range(len(sample)):
            if n % 2 == 1:
                fishlength = int(sample[n-1])
                fishcount = int(sample[n])
                for c in range(fishcount):
                    sample_lengths.append(fishlength)
        fish_lengths.append(sample_lengths)
    return fish_lengths

#input string data series with rows of "[1,1,2,2,2,3,4,7,7,7]"
#returns fish_count series column as a list of float averages e.g. [5.1, 4.6, 8.8]
def fish_length_averager(dfseries):
    fish_length_averages = []
    for row in dfseries:
        sample = [int(x) for x in row if x != 0]
        if bool(sample) == True:
            fish_length_averages.append(np.mean(sample))
        else:
            fish_length_averages.append(np.nan)
    return fish_length_averages

#input quartile data series
#split lower quartile and convert to float
def lower_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[0]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list

#input quartile data series
#split upper quartile and convert to float
def upper_quartile_cleaner(dseries):
    quartile_list = []
    for quartile in dseries:
        try:
            if len(quartile.split("-")) == 2:
                try:
                    quartile_list.append(float(quartile.split("-")[1]))
                except ValueError:
                    quartile_list.append(float("nan"))
                    pass
            else:
                quartile_list.append(float("nan"))
        except AttributeError:
            quartile_list.append(quartile)
            pass
    return quartile_list



In [4]:
lake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5422 entries, 0 to 5421
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   lake_id             5422 non-null   object 
 1   lake_name           5422 non-null   object 
 2   lake_county         5422 non-null   object 
 3   nearest_town        5008 non-null   object 
 4   lake_coordinates    5422 non-null   object 
 5   lake_area           5422 non-null   float64
 6   littoral_area       5422 non-null   float64
 7   lake_depth          5422 non-null   float64
 8   mean_depth          5422 non-null   float64
 9   water_access_sites  5422 non-null   object 
dtypes: float64(4), object(6)
memory usage: 423.7+ KB


In [5]:
#add lat and lon columns and drop coordinates column
lake = lake.drop_duplicates()
lake_coordinates = [json.loads(x) for x in lake["lake_coordinates"]]
longitude = [x[0] for x in lake_coordinates]
latitude = [x[1] for x in lake_coordinates]
lake["lake_longitude"] = longitude
lake["lake_latitude"] = latitude
lake = lake.drop(columns=["lake_coordinates"])
lake.head()

,lake_id,lake_name,lake_county,nearest_town,lake_area,littoral_area,lake_depth,mean_depth,water_access_sites,lake_longitude,lake_latitude
0,01004000,Aitkin,Aitkin,Libby,658.82,453.0,35.0,0.0,[],-93.289871,46.804042
1,01003100,Anderson,Aitkin,Sheshebee,91.34,83.0,16.0,3.0,[],-93.232645,46.737721
2,01016900,Bachelor,Aitkin,Aitkin,53.56,30.0,47.0,12.0,[],-93.726840,46.473611
3,01004600,Ball Bluff,Aitkin,Jacobson,168.64,32.0,78.0,25.0,['was01599'],-93.280365,46.932611
4,01006300,Bass,Aitkin,McGregor,89.13,55.0,42.0,0.0,[],-93.310700,46.808330


In [6]:
lake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5422 entries, 0 to 5421
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   lake_id             5422 non-null   object 
 1   lake_name           5422 non-null   object 
 2   lake_county         5422 non-null   object 
 3   nearest_town        5008 non-null   object 
 4   lake_area           5422 non-null   float64
 5   littoral_area       5422 non-null   float64
 6   lake_depth          5422 non-null   float64
 7   mean_depth          5422 non-null   float64
 8   water_access_sites  5422 non-null   object 
 9   lake_longitude      5422 non-null   float64
 10  lake_latitude       5422 non-null   float64
dtypes: float64(6), object(5)
memory usage: 508.3+ KB


In [7]:
was.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3013 entries, 0 to 3012
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   was_site_id                           3013 non-null   object
 1   was_site_name                         3013 non-null   object
 2   directions_to_was_site                3013 non-null   object
 3   was_site_coordinates                  3013 non-null   object
 4   was_site_administrator                3010 non-null   object
 5   number_of_docks                       3013 non-null   int64 
 6   number_of_restrooms                   3013 non-null   int64 
 7   number_of_parking_lots                3013 non-null   int64 
 8   number_of_handicap_accessible_spaces  3013 non-null   int64 
 9   number_of_trailer_spaces              3013 non-null   int64 
 10  number_of_vehicle_spaces              3013 non-null   int64 
 11  parking_lot_surface_type      

In [8]:
#add lat lon columns and drop coordinates column
was = was.drop_duplicates()
site_coordinates = [json.loads(x) for x in was["was_site_coordinates"]]
longitude = [x[0] for x in site_coordinates]
latitude = [x[1] for x in site_coordinates]
was["was_site_longitude"] = longitude
was["was_site_latitude"] = latitude
was = was.drop(columns=["was_site_coordinates"])
was.head()

,was_site_id,was_site_name,directions_to_was_site,was_site_administrator,number_of_docks,number_of_restrooms,number_of_parking_lots,number_of_handicap_accessible_spaces,number_of_trailer_spaces,number_of_vehicle_spaces,parking_lot_surface_type,number_of_launch_ramps,launch_ramp_surface_type,was_site_longitude,was_site_latitude
0,was00001,South Fork Kawishiwi River (S) Public Water Ac...,"From Ely drive 10.4 miles south on Hwy 1, turn...",U.S. Forest Service,1,1,1,0,7,7,gravel,1,concrete slab,-91.733457,47.816558
1,was00002,"Rainy Lake, Ranier Public Water Access Site",In the City of Ranier; launch ramp is Spruce S...,City of Ranier,1,0,1,0,8,0,NaN,1,concrete slab,-93.348122,48.615995
2,was00003,Severance Lake Public Water Access Site,Head east out of New Auburn for 7 miles to the...,DNR Division of Fish and Wildlife,0,0,0,0,0,0,NaN,0,NaN,-94.132386,44.675798
3,was00004,"Redwood River, Wabasso Public Water Access Site",On Hwy 6 between 325th St and 310th St,DNR Division of Parks and Trails,0,0,1,0,10,0,natural,1,concrete slab,-95.229799,44.507897
4,was00005,Shetek Lake Marsh's Landing Public Water Acces...,From US-59 follow Valhalla Rd east for 1.5 mil...,Murray County,2,1,1,0,15,5,gravel,2,concrete slab,-95.713395,44.108909


In [9]:
was.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3013 entries, 0 to 3012
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   was_site_id                           3013 non-null   object 
 1   was_site_name                         3013 non-null   object 
 2   directions_to_was_site                3013 non-null   object 
 3   was_site_administrator                3010 non-null   object 
 4   number_of_docks                       3013 non-null   int64  
 5   number_of_restrooms                   3013 non-null   int64  
 6   number_of_parking_lots                3013 non-null   int64  
 7   number_of_handicap_accessible_spaces  3013 non-null   int64  
 8   number_of_trailer_spaces              3013 non-null   int64  
 9   number_of_vehicle_spaces              3013 non-null   int64  
 10  parking_lot_surface_type              2718 non-null   object 
 11  number_of_launch_

In [10]:
length.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108078 entries, 0 to 108077
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   lake_id         108078 non-null  object
 1   species         108078 non-null  object
 2   fish_count      108078 non-null  object
 3   maximum_length  108078 non-null  int64 
 4   minimum_length  108078 non-null  int64 
 5   survey_id       108078 non-null  object
 6   survey_date     108078 non-null  object
dtypes: int64(2), object(5)
memory usage: 5.8+ MB


In [11]:
#clean up the length file duplicates, clean dates, convert lengths in count to histogram, and create average column
length = length.drop_duplicates()
length["survey_date"] = survey_date_cleaner(length["survey_date"])
length["fish_count"] = fish_length_cleaner(length["fish_count"])
length["average_length"] = fish_length_averager(length["fish_count"])
length = length.sort_values("survey_date")
length.head()

,lake_id,species,fish_count,maximum_length,minimum_length,survey_id,survey_date,average_length
88821,62006700,CAP,[0],0,0,120236,1953-09-01,NaN
88822,62006700,BLC,[0],0,0,120236,1953-09-01,NaN
88823,62006700,BLG,[0],0,0,120236,1953-09-01,NaN
88824,62006700,WTS,[0],0,0,120236,1953-09-01,NaN
88825,62006700,GSF,[0],0,0,120236,1953-09-01,NaN


In [12]:
length.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108078 entries, 88821 to 62486
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   lake_id         108078 non-null  object        
 1   species         108078 non-null  object        
 2   fish_count      108078 non-null  object        
 3   maximum_length  108078 non-null  int64         
 4   minimum_length  108078 non-null  int64         
 5   survey_id       108078 non-null  object        
 6   survey_date     108078 non-null  datetime64[ns]
 7   average_length  107890 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 7.4+ MB


In [13]:
catch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330239 entries, 0 to 330238
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   lake_id          330239 non-null  object 
 1   cpue             330239 non-null  object 
 2   average_weight   330220 non-null  float64
 3   gear             330239 non-null  object 
 4   gear_count       330239 non-null  float64
 5   quartile_count   212040 non-null  object 
 6   quartile_weight  210167 non-null  object 
 7   species          330239 non-null  object 
 8   survey_id        330239 non-null  object 
 9   survey_date      330239 non-null  object 
 10  total_catch      330239 non-null  int64  
 11  total_weight     330239 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 30.2+ MB


In [14]:
#clean cpue and date and separate quartiles into columns.
catch = catch.drop_duplicates()
catch["cpue"] = cpue_cleaner(catch["cpue"])
catch["survey_date"] = survey_date_cleaner(catch["survey_date"])
catch["count_lower_quartile"] = lower_quartile_cleaner(catch["quartile_count"])
catch["count_upper_quartile"] = upper_quartile_cleaner(catch["quartile_count"])
catch["weight_lower_quartile"] = lower_quartile_cleaner(catch["quartile_weight"])
catch["weight_upper_quartile"] = upper_quartile_cleaner(catch["quartile_weight"])
catch["gear_count"] = catch["gear_count"].astype("int")
catch = catch.drop(columns=["quartile_count", "quartile_weight"])
catch = catch.sort_values("survey_date")

2366 ? valueerror
2379 ? valueerror
2389 ? valueerror
2396 ? valueerror
2416 ? valueerror
2769 ? valueerror
4904 ? valueerror
5536 ? valueerror
9668 ? valueerror
10593 ? valueerror
11739 ? valueerror
15158 ? valueerror
15159 ? valueerror
15312 ? valueerror
15314 ? valueerror
15322 ? valueerror
15361 ? valueerror
15375 ? valueerror
15380 ? valueerror
15383 ? valueerror
15386 ? valueerror
15388 ? valueerror
15499 ? valueerror
15950 ? valueerror
15965 ? valueerror
34239 ? valueerror
34242 ? valueerror
34252 ? valueerror
34254 ? valueerror
48274 ? valueerror
49782 ? valueerror
50996 ? valueerror
51146 ? valueerror
52287 ? valueerror
61936 ? valueerror
67008 ? valueerror
67010 ? valueerror
67014 ? valueerror
67016 ? valueerror
67017 ? valueerror
67019 ? valueerror
67020 ? valueerror
67023 ? valueerror
67024 ? valueerror
74976 ? valueerror
85585 ? valueerror
85711 ? valueerror
85743 ? valueerror
100058 ? valueerror
100081 ? valueerror
100183 ? valueerror
100268 ? valueerror
100271 ? valueerr

In [15]:
catch.head()

,lake_id,cpue,average_weight,gear,gear_count,species,survey_id,survey_date,total_catch,total_weight,count_lower_quartile,count_upper_quartile,weight_lower_quartile,weight_upper_quartile
55596,10004200,209.50,0.17,Standard gill nets,2,BLB,121577,1940-06-24,419,71.5,7.7,104.7,0.2,0.5
55602,10004200,0.25,1.50,Standard trap nets,4,WTS,121577,1940-06-24,1,1.5,0.2,1.2,1.3,2.5
55597,10004200,16.50,0.15,Standard gill nets,2,YEP,121577,1940-06-24,33,4.8,2.0,22.3,0.1,0.2
55595,10004200,66.00,0.22,Standard gill nets,2,BLC,121577,1940-06-24,132,29.0,1.7,17.5,0.1,0.3
55594,10004200,0.50,4.50,Standard gill nets,2,BOF,121577,1940-06-24,1,4.5,0.3,1.3,2.0,4.4


In [16]:
catch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330239 entries, 55596 to 196747
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   lake_id                330239 non-null  object        
 1   cpue                   330028 non-null  float64       
 2   average_weight         330220 non-null  float64       
 3   gear                   330239 non-null  object        
 4   gear_count             330239 non-null  int32         
 5   species                330239 non-null  object        
 6   survey_id              330239 non-null  object        
 7   survey_date            330239 non-null  datetime64[ns]
 8   total_catch            330239 non-null  int64         
 9   total_weight           330239 non-null  float64       
 10  count_lower_quartile   211335 non-null  float64       
 11  count_upper_quartile   211335 non-null  float64       
 12  weight_lower_quartile  209521 non-null  

In [17]:
#create csv files to load into database
lake.to_csv("lake_info/lake_information_clean.csv", index=False)
length.to_csv("fish_length_info/fish_length_clean.csv", index=False)
catch.to_csv("fish_catch_info/fish_catch_clean.csv", index=False)

In [18]:
#add lake id to water access sites
was_values = [x.replace("\'","\"") for x in list(lake["water_access_sites"])]
was_values = [json.loads(x) for x in was_values]
lake["was_site_id"] = was_values
was_lake = lake.loc[lake["was_site_id"].str.len() > 0, ["lake_id", "lake_name", "was_site_id"]]
was_exploded = was_lake.explode("was_site_id")
was_exploded_clean = was_exploded.drop_duplicates(subset = "was_site_id")
was = was.merge(was_exploded_clean, on="was_site_id", how="left")
was.head(5)

,was_site_id,was_site_name,directions_to_was_site,was_site_administrator,number_of_docks,number_of_restrooms,number_of_parking_lots,number_of_handicap_accessible_spaces,number_of_trailer_spaces,number_of_vehicle_spaces,parking_lot_surface_type,number_of_launch_ramps,launch_ramp_surface_type,was_site_longitude,was_site_latitude,lake_id,lake_name
0,was00001,South Fork Kawishiwi River (S) Public Water Ac...,"From Ely drive 10.4 miles south on Hwy 1, turn...",U.S. Forest Service,1,1,1,0,7,7,gravel,1,concrete slab,-91.733457,47.816558,NaN,NaN
1,was00002,"Rainy Lake, Ranier Public Water Access Site",In the City of Ranier; launch ramp is Spruce S...,City of Ranier,1,0,1,0,8,0,NaN,1,concrete slab,-93.348122,48.615995,69069400,Rainy
2,was00003,Severance Lake Public Water Access Site,Head east out of New Auburn for 7 miles to the...,DNR Division of Fish and Wildlife,0,0,0,0,0,0,NaN,0,NaN,-94.132386,44.675798,72003000,Severance
3,was00004,"Redwood River, Wabasso Public Water Access Site",On Hwy 6 between 325th St and 310th St,DNR Division of Parks and Trails,0,0,1,0,10,0,natural,1,concrete slab,-95.229799,44.507897,NaN,NaN
4,was00005,Shetek Lake Marsh's Landing Public Water Acces...,From US-59 follow Valhalla Rd east for 1.5 mil...,Murray County,2,1,1,0,15,5,gravel,2,concrete slab,-95.713395,44.108909,51004600,Shetek
5,was00006,"Shetek Lake, Inlet Public Water Access Site",From US-59 follow 221 St east for 0.7 miles to...,DNR Division of Parks and Trails,0,0,1,0,0,0,gravel,1,concrete slab,-95.729775,44.167711,51004600,Shetek
6,was00007,St. James Lake (SE) Public Water Access Site,"Within the city limits of Saint James, off CSA...",City of St. James,0,1,1,0,0,0,asphalt,1,concrete slab,-94.640417,43.974702,83004300,St. James
7,was00008,Sulem Lake Public Water Access Site,3.5 mi SW of Butterfield to S shore.,DNR Division of Parks and Trails,0,0,1,0,0,10,natural,0,natural,-94.837386,43.926395,83005100,Sulem
8,was00009,"Watonwan River, Medelia Public Water Access Site",In Watona Park at on the south side of Madelia.,City of Madelia,0,0,1,0,10,10,gravel,1,concrete slab,-94.422457,44.039355,NaN,NaN
9,was00010,West Graham Lake (E) Public Water Access Site,"1 mile southwest of Kinbrae, on County Road 55...",Nobles County,0,0,1,0,10,6,gravel,1,concrete slab,-95.503410,43.808570,53002100,West Graham


In [19]:
#create csv file to load into database
was.to_csv("was_info/water_access_information_clean.csv", index=False)